In [16]:
import sys

import io
import os
import pickle
import random

from tqdm import tqdm

from time import sleep

import pandas as pd
import requests

from bs4 import BeautifulSoup

import json

In [ ]:
tbca_completo = None

with open("mapa_tbca_completo.pkl", "rb") as f:
    tbca_completo = pickle.load(f)
    
print(tbca_completo)